<a href="https://colab.research.google.com/github/MGASALUCAS/Notebooks-Collection/blob/main/ZindiChallenge1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## ZINDI CHALLENGE
# The challenge aim to develop a machine learning model to predict beavoiur of the customer to become inactive 
# and not making any transactions for 90 days

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Installing catboost  to boost decision tree during ML model development.

!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing the neccesary packages

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold
import re

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# loading the datasets.

train = pd.read_csv("Train.csv")
test  = pd.read_csv("Test.csv")
sample  = pd.read_csv("SampleSubmission.csv")
var = pd.read_csv('VariableDefinitions.csv')

In [ ]:
#Analyzing the submision file.

ss = pd.read_csv('SampleSubmission.csv')
ss.head()

,user_id,CHURN
0,51fe4c3347db1f8571d18ac03f716c41acee30a4,0
1,5ad5d67c175bce107cc97b98c4e37dcc38aa7f3e,0
2,5a4db591c953a8d8f373877fad37aaf4268899a1,0
3,8bf9b4d8880aeba1c9a0da48be78f12e629be37c,0
4,c7cdf2af01e9fa95bf498b68c122aa4b9a8d10df,0


In [ ]:
# Getting to know the variable definitions

vv = pd.read_csv('VariableDefinitions.csv')
vv.info

<bound method DataFrame.info of    Variable Definitions                                         Unnamed: 1  \
0                   NaN                                             French   
1                   NaN  Le dataset churn comprend 19 variables dont 15...   
2               user_id                                                NaN   
3                REGION                       la localité de chaque client   
4                TENURE                            la durée dans le reseau   
5               MONTANT                                montant de recharge   
6        FREQUENCE_RECH   nombre de fois que le client a fait une recharge   
7               REVENUE                    revenu mensuel de chaque client   
8          ARPU_SEGMENT                              revenu sur 90 jours/3   
9             FREQUENCE         nombre de fois que client à fait un revenu   
10          DATA_VOLUME                               nombre de connexions   
11               ON_NET         

In [ ]:
#showing other extra information about the dataset.

df1 = pd.read_csv("Train.csv")
print(df1.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077024 entries, 0 to 1077023
Data columns (total 19 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   user_id         1077024 non-null  object 
 1   REGION          652687 non-null   object 
 2   TENURE          1077024 non-null  object 
 3   MONTANT         699139 non-null   float64
 4   FREQUENCE_RECH  699139 non-null   float64
 5   REVENUE         714669 non-null   float64
 6   ARPU_SEGMENT    714669 non-null   float64
 7   FREQUENCE       714669 non-null   float64
 8   DATA_VOLUME     547261 non-null   float64
 9   ON_NET          683850 non-null   float64
 10  ORANGE          629880 non-null   float64
 11  TIGO            432250 non-null   float64
 12  ZONE1           84898 non-null    float64
 13  ZONE2           68794 non-null    float64
 14  MRG             1077024 non-null  object 
 15  REGULARITY      1077024 non-null  int64  
 16  TOP_PACK        626129 non-null   ob

In [ ]:
train.head(2)
train.shape

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,7ee9e11e342e27c70455960acc80d3f91c1286d1,DAKAR,K > 24 month,20000.0,47.0,21602.0,7201.0,52.0,8835.0,3391.0,396.0,185.0,NaN,NaN,NO,62,On net 200F=Unlimited _call24H,30.0,0
1,50443f42bdc92b10388fc56e520e4421a5fa655c,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,3,NaN,NaN,0


(1077024, 19)

In [ ]:
test.shape

(190063, 18)

In [ ]:
train.REGULARITY.nunique()

62

In [ ]:
train.isna().sum() / train.shape[0]

user_id           0.000000
REGION            0.393990
TENURE            0.000000
MONTANT           0.350860
FREQUENCE_RECH    0.350860
REVENUE           0.336441
ARPU_SEGMENT      0.336441
FREQUENCE         0.336441
DATA_VOLUME       0.491877
ON_NET            0.365056
ORANGE            0.415166
TIGO              0.598663
ZONE1             0.921174
ZONE2             0.936126
MRG               0.000000
REGULARITY        0.000000
TOP_PACK          0.418649
FREQ_TOP_PACK     0.418649
CHURN             0.000000
dtype: float64

In [ ]:
# FEATURE ENGINEERING FOR DATA MANIPULATION FOR BETTER ACCURACY AND PERFORMANCE

In [ ]:
useless = ['user_id', 'MRG']

In [ ]:
train.REGION.fillna('REG',inplace=True)
test.REGION.fillna('REG',inplace=True)

In [ ]:
train.TOP_PACK.fillna('PACK',inplace=True)
test.TOP_PACK.fillna('PACK',inplace=True)

In [ ]:
minus_999999 = ['MONTANT','FREQUENCE_RECH','REVENUE','ARPU_SEGMENT','FREQUENCE','DATA_VOLUME','ON_NET','ORANGE','FREQ_TOP_PACK','TIGO','ZONE1','ZONE2']

In [ ]:
for col in minus_999999:
    train[col].fillna(-99999, inplace=True)
    test[col].fillna(-99999, inplace=True)

In [ ]:
train.drop(columns=[i for i in useless], inplace=True)
test.drop(columns=[i for i in useless], inplace=True)

In [ ]:
data = pd.concat([train,test],axis=0)
data = data.reset_index(drop=True)

In [ ]:
def tenure(x):
    'function to encode the Tenure based on the lowest amount of month spent'
    if x == 'K > 24 month':
        return 24
    elif x == 'H 15-18 month':
        return 15
    elif x == 'G 12-15 month':
        return 12
    elif x == 'J 21-24 month':
        return 21
    elif x == 'I 18-21 month':
        return 18
    elif x == 'E 6-9 month':
        return 6
    elif x == 'F 9-12 month':
        return 9
    elif x == 'D 3-6 month':
        return 3
    else:
        pass

In [ ]:
data.TENURE = data.TENURE.apply(tenure)

In [ ]:
categ = ['REGION','TOP_PACK']

In [ ]:
len(data.REGION.unique()), len(data.TOP_PACK.unique())

(15, 129)

In [ ]:
data = pd.get_dummies(data, prefix = categ, columns = categ)

In [ ]:
data.head(3)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,TOP_PACK_VAS(IVR_Radio_Weekly),TOP_PACK_WIFI_ Family _10MBPS,TOP_PACK_WIFI_ Family _4MBPS,TOP_PACK_WIFI_Family_2MBPS,"TOP_PACK_YMGX 100=1 hour FNF, 24H/1 month","TOP_PACK_YMGX on-net 100=700F, 24H",TOP_PACK_Yewouleen_PKG,TOP_PACK_pack_chinguitel_24h,TOP_PACK_pilot_offer4,TOP_PACK_pilot_offer6
0,24,20000.0,47.0,21602.0,7201.0,52.0,8835.0,3391.0,396.0,185.0,...,0,0,0,0,0,0,0,0,0,0
1,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,0,0,0,0,0,0
2,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# FEATURE CREATION FOR ADDING NEW FEATURES TO IMPROVE PERFOMANCE.

In [ ]:
data['The income in Topup'] = data['REVENUE'] / data['MONTANT']
data['Active time'] =  data['TENURE'] / data['REGULARITY']
data['Revenue over income ratio'] = data['REVENUE'] / data['MONTANT']
data['Income left after Topup'] = data['REVENUE'] - data['MONTANT']
data['Tenure over frequency'] = data['TENURE'] / data['FREQUENCE_RECH']
data['Argument over revenue'] = data['ARPU_SEGMENT'] / data['REVENUE'] 

In [ ]:
train=data[data.CHURN.notnull()].reset_index(drop=True)
test=data[data.CHURN.isna()].reset_index(drop=True)

In [ ]:
y = train.CHURN
train.drop('CHURN', axis=1, inplace=True)
test.drop('CHURN', axis=1, inplace=True)

In [ ]:
test.head(2)
train.shape

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,TOP_PACK_Yewouleen_PKG,TOP_PACK_pack_chinguitel_24h,TOP_PACK_pilot_offer4,TOP_PACK_pilot_offer6,income_per_topup_amount,active_per_duration,income_to_amount_ratio,income_left_after_top_up,tenure_to_topup_frequency,income_in90days_perRevenue
0,18,2500.0,5.0,2500.0,833.0,5.0,0.0,64.0,70.0,-99999.0,...,0,0,0,0,1.0,0.514286,1.0,0.0,3.60000,0.3332
1,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,1.0,12.000000,1.0,0.0,-0.00024,1.0000


(1077024, 164)

In [ ]:
c = [i for i in train.columns 
     if train[i].dtype == object
     ]
len(c)

0

In [ ]:
train.shape, y.shape, test.shape

((1077024, 164), (1077024,), (190063, 164))

In [ ]:
train.head(5)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,TOP_PACK_Yewouleen_PKG,TOP_PACK_pack_chinguitel_24h,TOP_PACK_pilot_offer4,TOP_PACK_pilot_offer6,income_per_topup_amount,active_per_duration,income_to_amount_ratio,income_left_after_top_up,tenure_to_topup_frequency,income_in90days_perRevenue
0,24,20000.0,47.0,21602.0,7201.0,52.0,8835.0,3391.0,396.0,185.0,...,0,0,0,0,1.080100,0.387097,1.080100,1602.0,0.510638,0.333349
1,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,1.000000,8.000000,1.000000,0.0,-0.000240,1.000000
2,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,0,0,0,0,1.000000,24.000000,1.000000,0.0,-0.000240,1.000000
3,24,7900.0,19.0,7896.0,2632.0,25.0,9385.0,27.0,46.0,20.0,...,0,0,0,0,0.999494,0.393443,0.999494,-4.0,1.263158,0.333333
4,24,12350.0,21.0,12351.0,4117.0,29.0,9360.0,66.0,102.0,34.0,...,0,0,0,0,1.000081,0.428571,1.000081,1.0,1.142857,0.333333


In [ ]:
clust = [
'REVENUE',
'MONTANT',
'FREQUENCE_RECH',
'ARPU_SEGMENT',
'FREQUENCE',
'ON_NET',
'ORANGE',
'REGULARITY',
'FREQ_TOP_PACK',
'REGION_REG',
'TOP_PACK_PACK']

In [ ]:
kmeans = KMeans(
        n_clusters=2, init='k-means++',
        n_init=20, max_iter=300,
        tol=1e-04, random_state=0
    )

In [ ]:
%%capture 
#To avoid clumsy cell output

for col in clust:
    if train[col].dtype == object:
        dummies = pd.get_dummies(train[col], drop_first=True)
        test_dum = pd.get_dummies(test[col], drop_first=True)
        kmeans.fit(dummies)
        train[col+'_grouped'] = kmeans.labels_
        test[col+'_grouped'] = kmeans.predict(test_dum)
    else:
        kmeans.fit(pd.DataFrame(train[col]))
        train[col+'_grouped'] = kmeans.labels_
        test[col+'_grouped'] = kmeans.predict(pd.DataFrame(test[col]))

In [ ]:
train.shape, test.shape

((1077024, 175), (190063, 175))

In [ ]:
del data

In [ ]:
sum(train.isna().sum())

0

In [ ]:
km = KMeans(
        n_clusters=2, init='k-means++',
        n_init=20, max_iter=300,
        tol=1e-04, random_state=0
    )

In [ ]:
km.fit(train)

KMeans(n_clusters=2, n_init=20, random_state=0)

In [ ]:
train['groups'] = km.labels_
test['groups'] = km.predict(test)

In [ ]:
train.shape, y.shape, test.shape

((1077024, 176), (1077024,), (190063, 176))

In [ ]:
train.head(3)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,FREQUENCE_RECH_grouped,ARPU_SEGMENT_grouped,FREQUENCE_grouped,ON_NET_grouped,ORANGE_grouped,REGULARITY_grouped,FREQ_TOP_PACK_grouped,REGION_REG_grouped,TOP_PACK_PACK_grouped,groups
0,24,20000.0,47.0,21602.0,7201.0,52.0,8835.0,3391.0,396.0,185.0,...,0,0,0,0,0,0,0,0,0,0
1,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,1,1,1,1,1,1,1,1,1,1
2,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
X_1, X_2, y_1, y_2 = train_test_split(train, y, test_size = 0.01, random_state = 0)


In [ ]:
X_3, X_4, y_3, y_4 = train_test_split(X_2, y_2, test_size = 0.1, random_state = 0)

In [ ]:

X_3.shape, y_3.shape

((9693, 176), (9693,))

In [ ]:
val_model = CatBoostClassifier(random_seed = 10, n_estimators = 1000)

In [ ]:
%%capture 
#how the model is doing.

val_model.fit(X_3, y_3)

Learning rate set to 0.027174
0:	learn: 0.6504029	total: 59ms	remaining: 58.9s
1:	learn: 0.6085357	total: 66.7ms	remaining: 33.3s
2:	learn: 0.5735465	total: 75.6ms	remaining: 25.1s
3:	learn: 0.5422436	total: 83.5ms	remaining: 20.8s
4:	learn: 0.5111304	total: 91.6ms	remaining: 18.2s
5:	learn: 0.4857483	total: 99.6ms	remaining: 16.5s
6:	learn: 0.4613825	total: 108ms	remaining: 15.3s
7:	learn: 0.4400514	total: 115ms	remaining: 14.3s
8:	learn: 0.4228577	total: 123ms	remaining: 13.6s
9:	learn: 0.4074173	total: 131ms	remaining: 13s
10:	learn: 0.3913339	total: 139ms	remaining: 12.5s
11:	learn: 0.3779811	total: 147ms	remaining: 12.1s
12:	learn: 0.3669091	total: 155ms	remaining: 11.8s
13:	learn: 0.3580528	total: 164ms	remaining: 11.5s
14:	learn: 0.3480524	total: 179ms	remaining: 11.7s
15:	learn: 0.3386783	total: 188ms	remaining: 11.5s
16:	learn: 0.3310911	total: 197ms	remaining: 11.4s
17:	learn: 0.3252141	total: 205ms	remaining: 11.2s
18:	learn: 0.3192808	total: 213ms	remaining: 11s
19:	learn: 

In [ ]:
val_pred = val_model.predict_proba(X_4)[:, 1]

In [ ]:
log_loss(y_4, val_pred)

0.2556055020571949

In [ ]:
train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x), inplace=True)
test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x), inplace=True)

In [ ]:
train.head(2)

,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,...,FREQUENCE_RECH_grouped,ARPU_SEGMENT_grouped,FREQUENCE_grouped,ON_NET_grouped,ORANGE_grouped,REGULARITY_grouped,FREQ_TOP_PACK_grouped,REGION_REG_grouped,TOP_PACK_PACK_grouped,groups
0,24,20000.0,47.0,21602.0,7201.0,52.0,8835.0,3391.0,396.0,185.0,...,0,0,0,0,0,0,0,0,0,0
1,24,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,-99999.0,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
train.shape, test.shape, y.shape

((1077024, 176), (190063, 176), (1077024,))

In [ ]:
%%capture
# Here we anayze the model perfomance 
# NB :  The process may take a long time. After analyzng it for a while you may stop it and proceed running the next.

errcb1=[]
y_pred_totcb1=[]
fold=KFold(n_splits=25)#25
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m1  = CatBoostClassifier(n_estimators=1000,eval_metric='Logloss',random_seed= 10, use_best_model=True)
    m1.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m1.predict_proba(X_test)[:,1]
    print("err: ",log_loss(y_test,preds))
    errcb1.append((log_loss(y_test,preds)))
    p1 = m1.predict_proba(test)[:,1]
    y_pred_totcb1.append(p1)
    i+=1
np.mean(errcb1)

Learning rate set to 0.175991
0:	learn: 0.4636700	test: 0.4638613	test1: 0.4644133	best: 0.4644133 (0)	total: 415ms	remaining: 6m 54s
100:	learn: 0.2510557	test: 0.2511030	test1: 0.2528756	best: 0.2528690 (96)	total: 34s	remaining: 5m 2s
200:	learn: 0.2500003	test: 0.2500514	test1: 0.2527116	best: 0.2526788 (194)	total: 1m 4s	remaining: 4m 18s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2526788038
bestIteration = 194

Shrink model to first 195 iterations.
Learning rate set to 0.175991
0:	learn: 0.4641776	test: 0.4640571	test1: 0.4655963	best: 0.4655963 (0)	total: 345ms	remaining: 5m 44s
100:	learn: 0.2510036	test: 0.2510782	test1: 0.2554321	best: 0.2554321 (100)	total: 32.6s	remaining: 4m 50s
200:	learn: 0.2498775	test: 0.2499128	test1: 0.2551462	best: 0.2551408 (196)	total: 1m 4s	remaining: 4m 14s
300:	learn: 0.2491392	test: 0.2491869	test1: 0.2551016	best: 0.2550886 (295)	total: 1m 34s	remaining: 3m 39s
400:	learn: 0.2484765	test: 0.2485211	test1: 0.2551413	b

KeyboardInterrupt: ignored

In [ ]:
cat_sub = sample.copy()

In [ ]:
cat_sub.CHURN = np.mean(y_pred_totcb1, axis=0)

In [ ]:
cat_sub.to_csv('MgasaLucas1.csv', index = False)

In [ ]:
xgb_sub = sample.copy()

In [ ]:
%%capture
# Analyzing the model performance 
# Note the process may taka a while , After running for a little bit of a time you may interupt the procces and goes on.

errcb2=[]
y_pred_totcb2=[]
fold=KFold(n_splits=25)#25
i=1
for train_index, test_index in fold.split(train,y):
    print(str(i) + ' iter')
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = XGBClassifier(n_estimators=1000,random_seed = 10, use_best_model=True, eval_metric = 'logloss')
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=100,verbose=100)
    preds=m2.predict_proba(X_test)[:,1]
    print("err: ",log_loss(y_test,preds))
    errcb2.append((log_loss(y_test,preds)))
    p2 = m2.predict_proba(test)[:,1]
    y_pred_totcb2.append(p2)
    i+=1
np.mean(errcb2)

KeyboardInterrupt: ignored

In [ ]:
xgb_sub.CHURN = np.mean(y_pred_totcb2, axis=0)

In [ ]:
xgb_sub.to_csv('MgasaLucas2.csv', index = False)